In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Data Encoding

In [2]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
with open('../data.nosync/ebert_test_train_data.pkl', 'rb') as f:
    train_data, test_data = pickle.load(f)

In [4]:
train_data, valid_data = train_test_split(train_data, stratify=train_data['stars'], test_size=0.1, random_state=42)

X_train = train_data['review']
y_train = train_data['stars']
X_valid = valid_data['review']
y_valid = valid_data['stars']
X_test = test_data['review']
y_test = test_data['stars']

print(len(X_train), len(y_train), len(X_valid), len(y_valid), len(X_test), len(y_test))

lbl_enc = preprocessing.LabelEncoder()
y_encoded = lbl_enc.fit_transform(list(y_train) + list(y_valid) + list(y_test))
y_train_enc = y_encoded[0:len(y_train)]
y_valid_enc = y_encoded[len(y_train):len(y_train) + len(y_valid)]
y_test_enc = y_encoded[len(y_train) + len(y_valid):len(y_train) + len(y_valid) + len(y_test)]

# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(X_train) + list(X_valid))
xtrain_tfv =  tfv.transform(X_train) 
xvalid_tfv = tfv.transform(X_valid)

# from http://www-nlp.stanford.edu/data/glove.840B.300d.zip

embeddings_index = {}
missed_values = []
with open('../../glove.840B.300d.txt') as f:
    i = 0
    for line in tqdm(f):
        try:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except Exception as e:
            missed_values.append(values)
        i += 1
        
print("Found %s words vectors" % len(embeddings_index))

5204 5204 579 579 1928 1928


/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/_param_validation.py:594: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
2196017it [02:13, 16500.74it/s]

Found 2195884 words vectors


In [14]:
def sentence2vector(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w not in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    else:
        return v / np.sqrt((v ** 2).sum())

In [18]:
x_train_glove = [sentence2vector(x) for x in X_train]
x_valid_glove = [sentence2vector(x) for x in X_valid]

In [19]:
x_train_glove = np.array(xtrain_glove)
x_valid_glove = np.array(xvalid_glove)

In [5]:
set(train_data['stars']), set(test_data['stars'])

({0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0},
 {0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0})

# Deep Learning

In [42]:
from keras import utils
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Activation, Dropout, Embedding, BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.callbacks import EarlyStopping

In [21]:
scl = preprocessing.StandardScaler()
x_train_glove_scl = scl.fit_transform(x_train_glove)
x_valid_glove_scl = scl.fit_transform(y_train_glove)

In [22]:
y_train_enc = utils.to_categorical(y_train)
y_valid_enc = utils.to_categorical(y_valid)

In [29]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(5))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
model.fit(x_train_glove_scl, y=y_train_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(x_valid_glove_scl, y_valid_enc))

Epoch 1/5
82/82 [==============================] - 2s 7ms/step - loss: 1.6545 - val_loss: 1.1853
Epoch 2/5
82/82 [==============================] - 0s 4ms/step - loss: 1.1786 - val_loss: 1.1185
Epoch 3/5
82/82 [==============================] - 0s 4ms/step - loss: 1.0197 - val_loss: 1.0971
Epoch 4/5
82/82 [==============================] - 0s 4ms/step - loss: 0.9462 - val_loss: 1.0728
Epoch 5/5
82/82 [==============================] - 0s 4ms/step - loss: 0.8577 - val_loss: 1.0756


## Tokenize

In [31]:
from keras.preprocessing import sequence, text

In [34]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(X_train) + list(X_valid))
x_train_seq = token.texts_to_sequences(X_train)
x_valid_seq = token.texts_to_sequences(X_valid)

# zero pad the sequences
x_train_pad = sequence.pad_sequences(x_train_seq, maxlen=max_len)
x_valid_pad = sequence.pad_sequences(x_valid_seq, maxlen=max_len)

word_index = token.word_index

In [35]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████████████████████████████| 85752/85752 [00:00<00:00, 86024.37it/s]


In [39]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
model.fit(
    x_train_pad, 
    y=y_train_enc, 
    batch_size=512, 
    epochs=100, 
    verbose=1, 
    validation_data=(x_valid_pad, y_valid_enc)
)

Epoch 1/100
11/11 [==============================] - 9s 587ms/step - loss: 1.3968 - val_loss: 1.3462
Epoch 2/100
11/11 [==============================] - 7s 662ms/step - loss: 1.3359 - val_loss: 1.3213
Epoch 3/100
11/11 [==============================] - 8s 762ms/step - loss: 1.3296 - val_loss: 1.3297
Epoch 4/100
11/11 [==============================] - 8s 721ms/step - loss: 1.3188 - val_loss: 1.3188
Epoch 5/100
11/11 [==============================] - 7s 672ms/step - loss: 1.3182 - val_loss: 1.3122
Epoch 6/100
11/11 [==============================] - 8s 685ms/step - loss: 1.3037 - val_loss: 1.3065
Epoch 7/100
11/11 [==============================] - 8s 685ms/step - loss: 1.3018 - val_loss: 1.3043
Epoch 8/100
11/11 [==============================] - 10s 955ms/step - loss: 1.2939 - val_loss: 1.2910
Epoch 9/100
11/11 [==============================] - 7s 608ms/step - loss: 1.2794 - val_loss: 1.3004
Epoch 10/100
11/11 [==============================] - 7s 628ms/step - loss: 1.2660 - val_l

### Early-Stopping

In [43]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(x_train_pad, y=y_train_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_valid_pad, y_valid_enc), callbacks=[earlystop])

Epoch 1/100
11/11 [==============================] - 20s 1s/step - loss: 1.3988 - val_loss: 1.3350
Epoch 2/100
11/11 [==============================] - 17s 2s/step - loss: 1.3415 - val_loss: 1.3447
Epoch 3/100
11/11 [==============================] - 16s 1s/step - loss: 1.3269 - val_loss: 1.3335
Epoch 4/100
11/11 [==============================] - 18s 2s/step - loss: 1.3176 - val_loss: 1.3033
Epoch 5/100
11/11 [==============================] - 16s 1s/step - loss: 1.3059 - val_loss: 1.3070
Epoch 6/100
11/11 [==============================] - 15s 1s/step - loss: 1.2829 - val_loss: 1.2886
Epoch 7/100
11/11 [==============================] - 15s 1s/step - loss: 1.2711 - val_loss: 1.2998
Epoch 8/100
11/11 [==============================] - 17s 2s/step - loss: 1.2547 - val_loss: 1.2750
Epoch 9/100
11/11 [==============================] - 14s 1s/step - loss: 1.2523 - val_loss: 1.3098
Epoch 10/100
11/11 [==============================] - 15s 1s/step - loss: 1.2571 - val_loss: 1.2813
Epoch 11/

In [46]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(x_train_pad, y=y_train_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_valid_pad, y_valid_enc), callbacks=[earlystop])

Epoch 1/100
11/11 [==============================] - 35s 2s/step - loss: 1.4112 - val_loss: 1.3360
Epoch 2/100
11/11 [==============================] - 28s 2s/step - loss: 1.3528 - val_loss: 1.3368
Epoch 3/100
11/11 [==============================] - 27s 2s/step - loss: 1.3327 - val_loss: 1.3362
Epoch 4/100
11/11 [==============================] - 26s 2s/step - loss: 1.3220 - val_loss: 1.3454


In [48]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(x_train_pad, y=y_train_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(x_valid_pad, y_valid_enc), callbacks=[earlystop])

Epoch 1/100
11/11 [==============================] - 33s 2s/step - loss: 1.4081 - val_loss: 1.3450
Epoch 2/100
11/11 [==============================] - 27s 2s/step - loss: 1.3437 - val_loss: 1.3448
Epoch 3/100
11/11 [==============================] - 28s 3s/step - loss: 1.3361 - val_loss: 1.3528
Epoch 4/100
11/11 [==============================] - 26s 2s/step - loss: 1.3255 - val_loss: 1.3625
Epoch 5/100
11/11 [==============================] - 30s 3s/step - loss: 1.3154 - val_loss: 1.3485


# Ensembling

In [49]:
# this is the main ensembling class. how to use it is in the next cell!
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        Ensembler init function
        :param model_dict: model dictionary, see README for its format
        :param num_folds: the number of folds for ensembling
        :param task_type: classification or regression
        :param optimize: the function to optimize for, e.g. AUC, logloss, etc. Must have two arguments y_test and y_pred
        :param lower_is_better: is lower value of optimization function better or higher
        :param save_path: path to which model pickles will be dumped to along with generated predictions, or None
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: training data in tabular format
        :param y: binary, multi-class or regression
        :return: chain of models to be used in prediction
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict

### Count Vectorizer

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(X_train) + list(X_valid))
xtrain_ctv =  ctv.transform(X_train) 
xvalid_ctv = ctv.transform(X_valid)

## Ensemble Training

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from nep_loss import multiclass_logloss
from sklearn.metrics import accuracy_score

In [ ]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, y_train, lentrain=x_train_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=x_valid_glove.shape[0])

[16:34:55] INFO Found 9 classes
[16:34:55] INFO Training Level 0 Fold # 1. Model # 0
[16:35:13] INFO Predicting Level 0. Fold # 1. Model # 0
[16:35:14] INFO Level 0. Fold # 1. Model # 0. Validation Score = 1.732681
[16:35:14] INFO Training Level 0 Fold # 2. Model # 0
[16:35:33] INFO Predicting Level 0. Fold # 2. Model # 0
[16:35:33] INFO Level 0. Fold # 2. Model # 0. Validation Score = 1.726837
[16:35:33] INFO Training Level 0 Fold # 3. Model # 0
[16:35:49] INFO Predicting Level 0. Fold # 3. Model # 0
[16:35:49] INFO Level 0. Fold # 3. Model # 0. Validation Score = 1.727804
[16:35:49] INFO Level 0. Model # 0. Mean Score = 1.729107. Std Dev = 0.002558
[16:35:49] INFO Training Level 0 Fold # 1. Model # 1


/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[16:43:59] INFO Predicting Level 0. Fold # 1. Model # 1
[16:43:59] INFO Level 0. Fold # 1. Model # 1. Validation Score = 1.844908
[16:43:59] INFO Training Level 0 Fold # 2. Model # 1


/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[16:52:40] INFO Predicting Level 0. Fold # 2. Model # 1
[16:52:40] INFO Level 0. Fold # 2. Model # 1. Validation Score = 1.814251
[16:52:40] INFO Training Level 0 Fold # 3. Model # 1


...

/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [17:00:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[17:01:01] INFO Predicting Level 1. Fold # 2. Model # 0
[17:01:01] INFO Level 1. Fold # 2. Model # 0. Validation Score = 1.974962
[17:01:01] INFO Training Level 1 Fold # 3. Model # 0


/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [17:01:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[17:01:06] INFO Predicting Level 1. Fold # 3. Model # 0
[17:01:06] INFO Level 1. Fold # 3. Model # 0. Validation Score = 1.891059
[17:01:06] INFO Level 1. Model # 0. Mean Score = 1.974101. Std Dev = 0.067454
[17:01:06] INFO Saving predictions for level # 1
[17:01:06] INFO Training Fulldata Level 0. Model # 0
[17:01:34] INFO Predicting Test Level 0. Model # 0
[17:01:34] INFO Training Fulldata Level 0. Model # 1


/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# check error:
multiclass_logloss(y_valid, preds[1])

In [ ]:
accuracy_score(y_valid, preds[1])